In [2]:
# import libraries
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

## Data Acquisition
www.baseball-reference.com only allows 20 requests per minute. If we violate this rule, our session will be locked out for one hour.

### Web Crawling and Scraping - World Series Winners and Runners Up

In [114]:
main_url = 'https://www.baseball-reference.com'
world_series_url = 'https://www.baseball-reference.com/postseason/world-series.shtml'
ws_url_page = requests.get(world_series_url)
ws_soup = BeautifulSoup(ws_url_page.content)

In [115]:
table = ws_soup.find_all('table', {'id': 'world_series_winners_al_nl'})

world_series_winner_links = []
world_series_ru_links = []
world_series_winner = []
world_series_ru = []
years1 = [f'200{str(i)}' for i in range(0,10)]
years2 = [f'201{str(i)}' for i in range(0,10)]
years3 = [f'202{str(i)}' for i in range(0,3)]
years4 = years1 + years2 + years3
years = [years4[-1-i] for i in range(len(years4))]

for t in table:
    td = t.find_all('td')
    for team in td:
        winner = team.find('strong')
        check1 = team.get('data-stat')
        if winner == None:
            if (check1 == 'al_winner') or (check1 == 'nl_winner'):
                try:
                    ru_href = team.find('a')['href']
                    world_series_ru_links.append(str(ru_href))
                    ru_string = str(ru_href)
                    world_series_ru.append(ru_string[7:10])
                    if ru_string[11:15] == '2000':
                        break
                except:
                    pass
        else:
            try:
                winner_href = winner.find('a')['href']
                world_series_winner_links.append(str(winner_href))
                winner_string = str(winner_href)
                world_series_winner.append(winner_string[7:10])
            except:
                pass

In [116]:
print(len(world_series_ru))
print(len(world_series_winner))
print(world_series_winner)
print(world_series_ru)

23
23
['HOU', 'ATL', 'LAD', 'WSN', 'BOS', 'HOU', 'CHC', 'KCR', 'SFG', 'BOS', 'SFG', 'STL', 'SFG', 'NYY', 'PHI', 'BOS', 'STL', 'CHW', 'BOS', 'FLA', 'ANA', 'ARI', 'NYY']
['PHI', 'HOU', 'TBR', 'HOU', 'LAD', 'LAD', 'CLE', 'NYM', 'KCR', 'STL', 'DET', 'TEX', 'TEX', 'PHI', 'TBR', 'COL', 'DET', 'HOU', 'STL', 'NYY', 'SFG', 'NYY', 'NYM']


In [126]:
# get the links to team game logs
win_team_bat_game_logs_links = [f'/teams/tgl.cgi?team={world_series_winner[i]}&t=b&year={years[i]}' for i in range(len(years)) if years[i] !='2020']
win_team_pitch_game_logs_links = [f'/teams/tgl.cgi?team={world_series_winner[i]}&t=p&year={years[i]}' for i in range(len(years)) if years[i] !='2020']
ru_team_bat_game_logs_links = [f'/teams/tgl.cgi?team={world_series_ru[i]}&t=b&year={years[i]}' for i in range(len(years)) if years[i] !='2020']
ru_team_pitch_game_logs_links = [f'/teams/tgl.cgi?team={world_series_ru[i]}&t=p&year={years[i]}' for i in range(len(years)) if years[i] !='2020']

win_team_batting_dataframes = []
win_team_pitching_dataframes = []
ru_team_batting_dataframes = []
ru_team_pitching_dataframes = []

for i in range(len(years)-1):
    if (i > 0) & (i%4 == 0):
        time.sleep(60)
    win_team_batting_dataframes.append(pd.read_html(f'{main_url}{win_team_bat_game_logs_links[i]}'))
    win_team_pitching_dataframes.append(pd.read_html(f'{main_url}{win_team_pitch_game_logs_links[i]}'))  
    ru_team_batting_dataframes.append(pd.read_html(f'{main_url}{ru_team_bat_game_logs_links[i]}'))
    ru_team_pitching_dataframes.append(pd.read_html(f'{main_url}{ru_team_pitch_game_logs_links[i]}'))

In [129]:
print(len(win_team_batting_dataframes))
win_team_batting_dataframes[21][0]

22


,Rk,Gtm,Date,Unnamed: 3,Opp,Rslt,PA,AB,R,H,...,SB,CS,BA,OBP,SLG,OPS,LOB,#,Thr,Opp. Starter (GmeSc)
0,1,1,Apr 3,@,ANA,"W,3-2",35,32,3,6,...,0,2,.188,.257,.375,.632,5,10,R,K.Hill(51)
1,2,2,Apr 4,@,ANA,"W,5-3",41,35,5,9,...,1,0,.224,.316,.418,.734,9,10,R,K.Bottenfield(52)
2,3,3,Apr 5,@,ANA,"L,6-12",40,40,6,13,...,1,0,.262,.319,.467,.786,7,11,L,S.Schoeneweis(36)
3,4,4,Apr 7,@,SEA,"L,5-7",35,35,5,9,...,0,0,.261,.305,.486,.791,3,12,L,J.Halama(52)
4,5,5,Apr 8,@,SEA,"W,3-2",40,34,3,10,...,1,0,.267,.321,.494,.815,10,10,R,G.Meche(52)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,157,157,Sep 27,@,TBD,"L,1-11",35,33,1,7,...,1,0,.278,.355,.453,.809,7,9,R,C.Lidle(65)
162,158,158,Sep 28,@,TBD,"L,3-11",36,34,3,10,...,1,1,.278,.355,.453,.809,6,11,R,B.Rekar(60)
163,159,159,Sep 29,@,BAL,"L,2-13",36,31,2,6,...,0,0,.278,.355,.452,.807,7,13,L,C.McElroy(63)
164,160,160,Sep 30,@,BAL,"L,1-9",35,34,1,7,...,1,0,.277,.354,.451,.805,7,12,R,M.Mussina(63)


In [128]:
win_team_pitching_dataframes[21][1]

,Rk,Gtm,Date,Unnamed: 3,Opp,Rslt,IP,H,R,ER,...,2B,3B,IBB,SH,SF,ROE,GDP,#,Umpire,Pitchers Used (Rest-GameScore-Dec)
0,1,1,Apr 3,@,ANA,"W,3-2",9.0,10,2,2,...,1,0,0,0,0,0,0,3,Tim McClelland,"O.Hernandez (99-58-W), J.Nelson (99-H), M.Rive..."
1,2,2,Apr 4,@,ANA,"W,5-3",9.0,10,3,0,...,0,0,0,0,0,1,2,4,Terry Craft,"R.Clemens (99-54), M.Stanton (99), R.Mendoza (..."
2,3,3,Apr 5,@,ANA,"L,6-12",8.0,12,12,12,...,4,0,0,0,2,0,0,4,Paul Schrieber,"D.Cone (99-14-L), A.Watson (99), T.Erdos (99),..."
3,4,4,Apr 7,@,SEA,"L,5-7",8.0,12,7,7,...,2,0,1,0,1,0,0,3,Tony Randazzo,"A.Pettitte (99-35-L), R.Mendoza (2), M.Stanton..."
4,5,5,Apr 8,@,SEA,"W,3-2",9.0,5,2,2,...,1,0,1,1,0,2,1,5,Ed Montague,"O.Hernandez (4-68), J.Grimsley (2), M.Stanton ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,157,157,Sep 27,@,TBD,"L,1-11",8.0,9,11,10,...,2,0,1,1,1,0,1,4,Jerry Layne,"D.Neagle (4-33-L), R.Keisler (9), J.Tessmer (1..."
162,158,158,Sep 28,@,TBD,"L,3-11",8.0,15,11,9,...,2,0,1,0,0,1,1,5,Ted Barrett,"R.Clemens (4-27-L), D.Gooden (2), R.Choate (2)..."
163,159,159,Sep 29,@,BAL,"L,2-13",8.0,13,13,13,...,0,0,0,0,0,1,0,4,Jerry Crawford,"A.Pettitte (4-4-L), D.Gooden (0), T.Lilly (3),..."
164,160,160,Sep 30,@,BAL,"L,1-9",8.0,12,9,9,...,4,0,0,0,0,1,2,4,Mike DiMuro,"D.Cone (4-23-L), R.Keisler (2), M.Rivera (5), ..."


### Data Cleaning

### Save Data